In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install required libraries
!pip install -q spacy scipy pandas numpy nltk

# Download the spaCy English model
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 135.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import os
dir = "/content/drive/MyDrive/"
output_dir = "/content/drive/MyDrive/Nour"
date = "june_2020"
data = f"{output_dir}/dedup_billgates_tweets_en_{date}.csv"

In [ ]:
import pandas as pd
df = pd.read_csv(data)

/tmp/ipython-input-1112041999.py:2: DtypeWarning: Columns (6,11,15,30,34) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data)


In [ ]:
print(df.columns)

Index(['id', 'tweet_url', 'created_at', 'user_screen_name', 'text',
       'tweet_type', 'coordinates', 'hashtags', 'media', 'urls',
       'favorite_count', 'in_reply_to_screen_name', 'in_reply_to_status_id',
       'in_reply_to_user_id', 'lang', 'place', 'possibly_sensitive',
       'retweet_count', 'retweet_or_quote_id', 'retweet_or_quote_screen_name',
       'retweet_or_quote_user_id', 'source', 'user_id', 'user_created_at',
       'user_default_profile_image', 'user_description',
       'user_favourites_count', 'user_followers_count', 'user_friends_count',
       'user_listed_count', 'user_location', 'user_name',
       'user_statuses_count', 'user_time_zone', 'user_urls', 'user_verified',
       'text_en', 'user_type'],
      dtype='object')


# **User metadata features**

In [ ]:
import numpy as np
from datetime import datetime
from scipy.stats import entropy, skew, kurtosis

def extract_user_metadata_features(df):
    # --- Ensure proper datetime format ---
    df['user_created_at'] = pd.to_datetime(df['user_created_at'], errors='coerce').dt.tz_localize(None)

    # --- Basic Profile Info ---
    df['screen_name_length'] = df['user_screen_name'].astype(str).apply(len)
    df['screen_name_digit_count'] = df['user_screen_name'].astype(str).apply(lambda x: sum(c.isdigit() for c in x))
    df['user_name_length'] = df['user_name'].astype(str).apply(len)
    df['account_age_days'] = (datetime(2021, 7, 8) - df['user_created_at']).dt.days
    df['default_profile_image'] = df['user_default_profile_image'].astype(int)
    df['user_verified'] = df['user_verified'].astype(int)

    # --- Profile Description Features ---
    df['user_description'] = df['user_description'].fillna('').astype(str)
    df['user_description_length'] = df['user_description'].apply(len)
    df['has_profile_description'] = (df['user_description_length'] > 0).astype(int)

    # Unique description count
    user_desc_df = df[['user_id', 'user_description']].drop_duplicates()
    desc_counts = user_desc_df['user_description'].value_counts()
    user_desc_df['description_occurrence_count'] = user_desc_df['user_description'].map(desc_counts)
    user_desc_df['is_description_unique'] = (user_desc_df['description_occurrence_count'] == 1).astype(int)
    df = df.merge(user_desc_df[['user_id', 'description_occurrence_count', 'is_description_unique']], on='user_id', how='left')

    # --- Distributional Statistics for Friends, Followers, Favorites, Tweets ---
    for col in ['user_friends_count', 'user_followers_count', 'user_favourites_count', 'user_statuses_count']:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

    distribution_stats = {}
    for col in ['user_friends_count', 'user_followers_count', 'user_favourites_count', 'user_statuses_count']:
        values = df[col].values
        distribution_stats[f'{col}_min'] = np.min(values)
        distribution_stats[f'{col}_max'] = np.max(values)
        distribution_stats[f'{col}_median'] = np.median(values)
        distribution_stats[f'{col}_mean'] = np.mean(values)
        distribution_stats[f'{col}_std'] = np.std(values)
        distribution_stats[f'{col}_skew'] = skew(values)
        distribution_stats[f'{col}_kurtosis'] = kurtosis(values)
        distribution_stats[f'{col}_entropy'] = entropy(np.histogram(values, bins=10, density=True)[0] + 1e-9)  # avoid log(0)

    # Convert to DataFrame for merging if needed
    stats_df = pd.DataFrame([distribution_stats])

    return df, stats_df


In [ ]:
# Re-import after kernel reset
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.stats import entropy, skew, kurtosis

def extract_user_metadata_features(df):
    # Ensure datetime parsing
    df['user_created_at'] = pd.to_datetime(df['user_created_at'], errors='coerce').dt.tz_localize(None)
    df['created_at'] = pd.to_datetime(df['created_at'], errors='coerce').dt.tz_localize(None)

    # Basic profile features
    df['screen_name_length'] = df['user_screen_name'].astype(str).apply(len)
    df['screen_name_digit_count'] = df['user_screen_name'].astype(str).apply(lambda x: sum(c.isdigit() for c in x))
    df['user_name_length'] = df['user_name'].astype(str).apply(len)
    df['account_age_days'] = (datetime(2021, 7, 8) - df['user_created_at']).dt.days
    df['default_profile_image'] = df['user_default_profile_image'].astype(int)
    df['user_verified'] = df['user_verified'].astype(int)

    # Description features
    df['user_description'] = df['user_description'].fillna('').astype(str)
    df['user_description_length'] = df['user_description'].apply(len)
    df['has_profile_description'] = (df['user_description_length'] > 0).astype(int)

    # Description uniqueness per user
    user_desc_df = df[['user_id', 'user_description']].drop_duplicates()
    desc_counts = user_desc_df['user_description'].value_counts()
    user_desc_df['description_occurrence_count'] = user_desc_df['user_description'].map(desc_counts)
    user_desc_df['is_description_unique'] = (user_desc_df['description_occurrence_count'] == 1).astype(int)
    df = df.merge(user_desc_df[['user_id', 'description_occurrence_count', 'is_description_unique']], on='user_id', how='left')

    # Distributional stats
    for col in ['user_friends_count', 'user_followers_count', 'user_favourites_count', 'user_statuses_count']:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

    distribution_stats = {}
    for col in ['user_friends_count', 'user_followers_count', 'user_favourites_count', 'user_statuses_count']:
        values = df[col].values
        distribution_stats[f'{col}_min'] = np.min(values)
        distribution_stats[f'{col}_max'] = np.max(values)
        distribution_stats[f'{col}_median'] = np.median(values)
        distribution_stats[f'{col}_mean'] = np.mean(values)
        distribution_stats[f'{col}_std'] = np.std(values)
        distribution_stats[f'{col}_skew'] = skew(values)
        distribution_stats[f'{col}_kurtosis'] = kurtosis(values)
        distribution_stats[f'{col}_entropy'] = entropy(np.histogram(values, bins=10, density=True)[0] + 1e-9)

    stats_df = pd.DataFrame([distribution_stats])

    # Aggregate tweet activity per user
    df['is_mention'] = df['text'].astype(str).apply(lambda x: 1 if '@' in x else 0)
    user_activity = df.groupby('user_id').agg(
        total_tweets=('tweet_type', lambda x: x.isin(['original', 'quote']).sum()),
        total_retweets=('tweet_type', lambda x: (x == 'retweet').sum()),
        total_replies=('tweet_type', lambda x: (x == 'reply').sum()),
        total_mentions=('is_mention', 'sum'),
        first_tweet_time=('created_at', 'min'),
        last_tweet_time=('created_at', 'max')
    ).reset_index()

    user_activity['active_hours'] = (user_activity['last_tweet_time'] - user_activity['first_tweet_time']).dt.total_seconds() / 3600
    user_activity['active_hours'] = user_activity['active_hours'].replace(0, np.nan)

    for col in ['tweets', 'retweets', 'replies', 'mentions']:
        user_activity[f'{col}_per_hour'] = user_activity[f'total_{col}'] / user_activity['active_hours']

    df = df.merge(user_activity[['user_id', 'total_tweets', 'tweets_per_hour',
                                 'total_retweets', 'retweets_per_hour',
                                 'total_replies', 'replies_per_hour',
                                 'total_mentions', 'mentions_per_hour']], on='user_id', how='left')

    return df, stats_df


In [ ]:
user_features, user_stats = extract_user_metadata_features(df)
print(user_features.head())
print(user_features.shape)
print(user_features.columns)

/tmp/ipython-input-2917691208.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['user_created_at'] = pd.to_datetime(df['user_created_at'], errors='coerce').dt.tz_localize(None)


                    id                                          tweet_url  \
0  1267443579045232651  https://twitter.com/IllinoisJack85/status/1267...   
1  1267520253904510976  https://twitter.com/IllinoisJack85/status/1267...   
2  1267587387443593217  https://twitter.com/IllinoisJack85/status/1267...   
3  1267840580299694082  https://twitter.com/IllinoisJack85/status/1267...   
4  1267887896884908032  https://twitter.com/IllinoisJack85/status/1267...   

           created_at user_screen_name  \
0 2020-06-01 13:11:11   IllinoisJack85   
1 2020-06-01 18:15:52   IllinoisJack85   
2 2020-06-01 22:42:38   IllinoisJack85   
3 2020-06-02 15:28:44   IllinoisJack85   
4 2020-06-02 18:36:45   IllinoisJack85   

                                                text tweet_type coordinates  \
0  During all those protests people suddenly forg...   original         NaN   
1  I just saw CBSN doing a POSITIVE report on Hyd...   original         NaN   
2  Who died and made you king? https://t.co/Phi

# **Timing features**

In [ ]:
df["in_reply_to_user_id"].unique()

array([           nan, 1.97906380e+07, 8.93248497e+17, ...,
       1.50858450e+07, 1.15419066e+18, 2.17993250e+07])

In [ ]:
df["user_id"].unique()

array(['IllinoisJack85', 'TheRISEofROD', 'GmanFan45', ..., 'VitaminSteve',
       'Kruzer60913071', 'marlahooch231'], dtype=object)

In [ ]:
len(df['created_at'].unique())

1169550

In [ ]:
len(df)

1716726

In [ ]:
df['tweet_type'].unique()

array(['original', 'retweet', 'quote', 'reply'], dtype=object)

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import entropy, skew, kurtosis

def compute_time_deltas(timestamps):
    timestamps = sorted(pd.to_datetime(timestamps, errors='coerce').dropna())
    if len(timestamps) < 2:
        return []
    deltas = [(t2 - t1).total_seconds() for t1, t2 in zip(timestamps[:-1], timestamps[1:])]
    return deltas

def extract_temporal_features(df):
    df['created_at'] = pd.to_datetime(df['created_at'], errors='coerce')

    user_features = []

    for user_id, group in df.groupby('user_id'):
        user_dict = {'user_id': user_id}

        tweet_times = group[group['tweet_type'] == 'original']['created_at']
        tweet_deltas = compute_time_deltas(tweet_times)

        retweet_times = group[group['tweet_type'] == 'retweet']['created_at']
        retweet_deltas = compute_time_deltas(retweet_times)

        mention_filter = group['text'].astype(str).str.contains('@') | group['in_reply_to_user_id'].notnull()
        mention_times = group[mention_filter]['created_at']
        mention_deltas = compute_time_deltas(mention_times)

        def compute_stats(prefix, deltas):
            if len(deltas) == 0:
                stats = {f"{prefix}_{s}": np.nan for s in ['min', 'max', 'median', 'mean', 'std', 'skew', 'kurtosis', 'entropy']}
            else:
                hist = np.histogram(deltas, bins=10, density=True)[0] + 1e-9
                stats = {
                    f"{prefix}_min": np.min(deltas),
                    f"{prefix}_max": np.max(deltas),
                    f"{prefix}_median": np.median(deltas),
                    f"{prefix}_mean": np.mean(deltas),
                    f"{prefix}_std": np.std(deltas),
                    f"{prefix}_skew": skew(deltas),
                    f"{prefix}_kurtosis": kurtosis(deltas),
                    f"{prefix}_entropy": entropy(hist)
                }
            return stats

        user_dict.update(compute_stats("tweet_time", tweet_deltas))
        user_dict.update(compute_stats("retweet_time", retweet_deltas))
        user_dict.update(compute_stats("mention_time", mention_deltas))

        user_features.append(user_dict)

    temporal_df = pd.DataFrame(user_features)
    return temporal_df


In [ ]:
temporal_features = extract_temporal_features(df)
print(temporal_features.head())
print(temporal_features.shape)
print(temporal_features.columns)

/tmp/ipython-input-741193436.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['created_at'] = pd.to_datetime(df['created_at'], errors='coerce')
/tmp/ipython-input-741193436.py:41: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  f"{prefix}_skew": skew(deltas),
/tmp/ipython-input-741193436.py:42: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  f"{prefix}_kurtosis": kurtosis(deltas),
/tmp/ipython-input-741193436.py:41: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  f"{prefix}

        user_id  tweet_time_min  tweet_time_max  tweet_time_median  \
0     00001745Q        141477.0        209983.0           175730.0   
1      0000DD02             NaN             NaN                NaN   
2   001_percent           133.0        872991.0            86791.0   
3     0045James             NaN             NaN                NaN   
4  0070American             NaN             NaN                NaN   

   tweet_time_mean  tweet_time_std  tweet_time_skew  tweet_time_kurtosis  \
0    175730.000000    34253.000000         0.000000            -2.000000   
1              NaN             NaN              NaN                  NaN   
2    154964.461538   231806.936052         2.179067             4.183867   
3              NaN             NaN              NaN                  NaN   
4              NaN             NaN              NaN                  NaN   

   tweet_time_entropy  retweet_time_min  ...  retweet_time_kurtosis  \
0            0.693816           56739.0  ...       

# **Content based features**

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
from scipy.stats import entropy, skew, kurtosis
import spacy

# Load spaCy English model
nlp = spacy.load("en_core_web_sm")

# Define POS categories as mapped in the paper
POS_TAGS = {
    "VERB": "verbs",
    "NOUN": "nouns",
    "ADJ": "adjectives",
    "AUX": "modals",           # Modal auxiliaries
    "DET": "predeterminers",  # Pre-determiners/determiners
    "INTJ": "interjections",
    "ADV": "adverbs",
    "PRON": "pronouns",
    "SCONJ": "wh_words",
    "ADP": "wh_words"  # Some wh-words
}

def get_pos_counts_and_props(doc):
    pos_counts = Counter()
    total = 0
    for token in doc:
        if token.pos_ in POS_TAGS:
            mapped = POS_TAGS[token.pos_]
            pos_counts[mapped] += 1
            total += 1
    pos_props = {tag: (pos_counts[tag] / total if total > 0 else 0) for tag in set(POS_TAGS.values())}
    return pos_counts, pos_props
def compute_distribution_stats(values):
    if len(values) == 0:
        return {stat: np.nan for stat in ['min', 'max', 'median', 'mean', 'std', 'skew', 'kurtosis', 'entropy']}
    values = np.array(values)
    hist = np.histogram(values, bins=10, density=True)[0] + 1e-9
    return {
        'min': np.min(values),
        'max': np.max(values),
        'median': np.median(values),
        'mean': np.mean(values),
        'std': np.std(values),
        'skew': skew(values),
        'kurtosis': kurtosis(values),
        'entropy': entropy(hist)
    }

def extract_content_language_features(df):
    user_features = []

    # Group once to avoid repeated calls
    grouped = df.groupby("user_id")

    for user_id, group in grouped:
        user_dict = {'user_id': user_id}
        tweet_texts = group['text_en'].dropna().astype(str).tolist()

        word_counts = []
        word_entropies = []
        pos_freqs_per_tweet = {tag: [] for tag in set(POS_TAGS.values())}
        pos_props_per_tweet = {tag: [] for tag in set(POS_TAGS.values())}

        # Use nlp.pipe for efficient bulk processing
        for doc in nlp.pipe(tweet_texts, disable=["ner"]):  # disabling NER for speed
            words = [token.text.lower() for token in doc if token.is_alpha]
            word_count = len(words)
            word_counts.append(word_count)

            # Word entropy
            word_freq = Counter(words)
            probs = np.array(list(word_freq.values())) / word_count if word_count > 0 else [0]
            word_entropies.append(entropy(probs) if word_count > 0 else 0)

            # POS tagging
            pos_counts, pos_props = get_pos_counts_and_props(doc)
            for tag in pos_freqs_per_tweet:
                pos_freqs_per_tweet[tag].append(pos_counts.get(tag, 0))
                pos_props_per_tweet[tag].append(pos_props.get(tag, 0))

        # Add distribution stats
        for stat_name, stat_value in compute_distribution_stats(word_counts).items():
            user_dict[f'word_count_{stat_name}'] = stat_value

        for stat_name, stat_value in compute_distribution_stats(word_entropies).items():
            user_dict[f'word_entropy_{stat_name}'] = stat_value

        for tag in pos_freqs_per_tweet:
            stats = compute_distribution_stats(pos_freqs_per_tweet[tag])
            for stat_name, stat_value in stats.items():
                user_dict[f'pos_freq_{tag}_{stat_name}'] = stat_value

        for tag in pos_props_per_tweet:
            stats = compute_distribution_stats(pos_props_per_tweet[tag])
            for stat_name, stat_value in stats.items():
                user_dict[f'pos_prop_{tag}_{stat_name}'] = stat_value

        user_features.append(user_dict)

    return pd.DataFrame(user_features)


In [ ]:
content_features = extract_content_language_features(df)
print(content_features.head())
print(content_features.columns)
print(content_features.shape)


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
/tmp/ipython-input-395660131.py:45: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  'skew': skew(values),
/tmp/ipython-input-395660131.py:46: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  'kurtosis': kurtosis(values),
/tmp/ipython-input-395660131.py:45: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  'skew': skew(values),
/tmp/ipython-input-395660131.py:46: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  'kurtosis': 

        user_id  word_count_min  word_count_max  word_count_median  \
0     00001745Q               6              55               10.0   
1      0000DD02               3              40               14.0   
2   001_percent               1              54               15.5   
3     0045James               7              43               21.5   
4  0070American               1              54               41.5   

   word_count_mean  word_count_std  word_count_skew  word_count_kurtosis  \
0        15.076923       13.831188         2.010434             2.806824   
1        14.133333        9.972406         1.000208             0.651721   
2        21.346154       14.930460         0.686959            -0.601884   
3        23.250000       12.852529         0.400289            -0.966921   
4        33.227273       16.533525        -0.566604            -1.203965   

   word_count_entropy  word_entropy_min  ...  pos_prop_pronouns_kurtosis  \
0            1.178655          1.560710  ...  

In [ ]:
content_features.isna().mean().sort_values(ascending=False).head(10)  # % of missing per column
content_features.describe().T[['mean', 'std']]                        # variability per feature

,mean,std
word_count_min,8.301659,9.173191
word_count_max,38.506414,14.029799
word_count_median,20.582778,9.761731
word_count_mean,21.587072,8.514716
word_count_std,9.561524,5.268500
...,...,...
pos_prop_nouns_mean,0.244454,0.083153
pos_prop_nouns_std,0.099700,0.065105
pos_prop_nouns_skew,0.361212,0.780891
pos_prop_nouns_kurtosis,0.267235,2.541813


In [ ]:
content_features.isna().sum().sort_values(ascending=False).head(10)


,0
pos_freq_interjections_kurtosis,28332
pos_freq_interjections_skew,28332
pos_prop_interjections_kurtosis,28278
pos_prop_interjections_skew,28278
pos_freq_adverbs_skew,10611
pos_freq_adverbs_kurtosis,10611
pos_prop_adverbs_kurtosis,9948
pos_prop_adverbs_skew,9948
pos_freq_adjectives_kurtosis,9820
pos_freq_adjectives_skew,9820


# **sentiment based features**

In [ ]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
vad_df = pd.read_csv(f"{dir}/Nour/lexicons/NRC-VAD-Lexicon-v2.1.txt", sep="\t", header=None, skiprows=1,names=["word", "valence", "arousal", "dominance"])

In [ ]:
vad_df.head()

,word,valence,arousal,dominance
0,a battery,0.134,-0.298,-0.096
1,a bit,-0.096,-0.264,-0.214
2,a bunch,0.088,-0.350,-0.068
3,a cappella,0.134,-0.116,-0.200
4,a couple,0.266,-0.110,0.090


In [ ]:
vad_df = vad_df.drop_duplicates(subset="word")
for col in ["valence", "arousal", "dominance"]:
    vad_df[col] = vad_df[col].astype(float)
vad_lexicon = vad_df.set_index("word")[["valence", "arousal", "dominance"]].to_dict("index")

In [ ]:
# Load Hedonometer lexicon
hedo_df = pd.read_csv(f"{dir}/Nour/lexicons/Hedonometer.csv")
hedo_df = hedo_df.dropna(subset=["Happiness Score"])
hedo_df["Happiness Score"] = hedo_df["Happiness Score"].astype(float)
hedo_lexicon = hedo_df.set_index("Word")["Happiness Score"].to_dict()

In [ ]:
import re
import pandas as pd
import numpy as np
from collections import Counter
from scipy.stats import entropy, skew, kurtosis
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Pre-compiled emoticon regex
POS_EMOS_RE = re.compile(r"(:\)|:D|:-\)|😊|😁|😄|😃|😆|☺️|😍|😘|🤩)")
NEG_EMOS_RE = re.compile(r"(:\(|:'\(|:-\(|😢|😭|😞|😔|😡|😠|🙁|☹️)")

# Initialize VADER
vader = SentimentIntensityAnalyzer()

def compute_distribution_stats(values):
    if len(values) == 0:
        return {stat: np.nan for stat in ['min', 'max', 'median', 'mean', 'std', 'skew', 'kurtosis', 'entropy']}
    values = np.array(values)
    hist = np.histogram(values, bins=10, density=True)[0] + 1e-9
    return {
        'min': np.min(values),
        'max': np.max(values),
        'median': np.median(values),
        'mean': np.mean(values),
        'std': np.std(values),
        'skew': skew(values),
        'kurtosis': kurtosis(values),
        'entropy': entropy(hist)
    }

def extract_sentiment_features(df):
    user_features = []

    for user_id, group in df.groupby("user_id"):
        user_dict = {"user_id": user_id}
        texts = group["text_en"].dropna().astype(str).tolist()

        happiness, valence, arousal, dominance = [], [], [], []
        polarization, pos_emo, neg_emo = [], [], []

        for text in texts:
            words = re.findall(r'\w+', text.lower())

            h_vals = [hedo_lexicon[w] for w in words if w in hedo_lexicon]
            v_vals = [vad_lexicon[w]["valence"] for w in words if w in vad_lexicon]
            a_vals = [vad_lexicon[w]["arousal"] for w in words if w in vad_lexicon]
            d_vals = [vad_lexicon[w]["dominance"] for w in words if w in vad_lexicon]

            happiness.append(np.mean(h_vals) if h_vals else 0)
            valence.append(np.mean(v_vals) if v_vals else 0)
            arousal.append(np.mean(a_vals) if a_vals else 0)
            dominance.append(np.mean(d_vals) if d_vals else 0)

            vader_score = vader.polarity_scores(text)
            polarization.append(abs(vader_score['pos'] - vader_score['neg']))

            pos_count = len(POS_EMOS_RE.findall(text))
            neg_count = len(NEG_EMOS_RE.findall(text))

            pos_emo.append(pos_count)
            neg_emo.append(neg_count)

        total_emo = np.array(pos_emo) + np.array(neg_emo)

        # Sentiment scores
        score_sets = {
            "happiness": happiness,
            "valence": valence,
            "arousal": arousal,
            "dominance": dominance,
            "polarization": polarization
        }

        for name, values in score_sets.items():
            stats = compute_distribution_stats(values)
            for stat, val in stats.items():
                user_dict[f"{name}_{stat}"] = val

        # Emoticon stats
        for label, values in {
            "pos_emoticons": pos_emo,
            "neg_emoticons": neg_emo,
            "total_emoticons": total_emo
        }.items():
            stats = compute_distribution_stats(values)
            for stat, val in stats.items():
                user_dict[f"{label}_{stat}"] = val

        # Ratios
        total_neg = sum(neg_emo)
        total_pos = sum(pos_emo)
        user_dict["pos_neg_emo_ratio"] = (total_pos / (total_neg + 1e-5)) if total_neg > 0 else np.nan
        user_dict["tweets_with_emoticons_ratio"] = (np.count_nonzero(total_emo) / len(texts)) if texts else 0

        user_features.append(user_dict)

    return pd.DataFrame(user_features)


In [ ]:
sentiment_features=extract_sentiment_features(df)
print(sentiment_features.head())
print(sentiment_features.columns)
print(sentiment_features.shape)

/tmp/ipython-input-3138955322.py:26: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  'skew': skew(values),
/tmp/ipython-input-3138955322.py:27: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  'kurtosis': kurtosis(values),
/tmp/ipython-input-3138955322.py:26: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  'skew': skew(values),
/tmp/ipython-input-3138955322.py:27: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  'kurtosis': kurtosis(values),
/usr/local/lib/python3.11/dist-packages/numpy/lib/_histo

        user_id  happiness_min  happiness_max  happiness_median  \
0     00001745Q       2.910000       5.481379          5.084444   
1      0000DD02       4.844118       6.840000          5.196000   
2   001_percent       2.340000       6.048571          5.208571   
3     0045James       4.835556       5.527500          5.056667   
4  0070American       2.780000       5.543488          5.189035   

   happiness_mean  happiness_std  happiness_skew  happiness_kurtosis  \
0        4.753309       0.703749       -1.412445            1.101334   
1        5.366057       0.539281        1.522946            1.522449   
2        5.165802       0.525525       -3.011389           13.914025   
3        5.119097       0.262654        0.564444           -1.160207   
4        5.121318       0.534828       -3.747882           13.738193   

   happiness_entropy  valence_min  ...  total_emoticons_min  \
0           1.585771       -0.164  ...                    0   
1           1.679204       -0.196  ...

In [ ]:
sentiment_features['total_emoticons_median'].unique()

array([ 0. ,  1. ,  0.5,  3. ,  2. ,  4. ,  1.5,  9. ,  5. , 39. ,  7. ,
        2.5,  6. ,  3.5])

# **network based features**

In [ ]:
!pip install networkx

In [ ]:
# Re-import necessary dependencies in case of reset
import pandas as pd
import numpy as np
import re
import networkx as nx
from collections import defaultdict
from scipy.stats import entropy, skew, kurtosis

MENTION_REGEX = r"@(\w{1,15})"

def build_user_interaction_graph(df):
    user_edges = defaultdict(list)
    for _, row in df.iterrows():
        author = row['user_screen_name']
        mentions = re.findall(MENTION_REGEX, str(row['text']))
        for mentioned_user in mentions:
            if mentioned_user != author:
                user_edges[author].append(mentioned_user)

        replied_to = row.get('in_reply_to_screen_name')
        if pd.notna(replied_to) and replied_to != author:
            user_edges[author].append(replied_to)

        quoted_user = row.get('retweet_or_quote_screen_name')
        if pd.notna(quoted_user) and quoted_user != author:
            user_edges[author].append(quoted_user)
    return user_edges

def compute_distribution_stats(values):
    if len(values) == 0:
        return {stat: np.nan for stat in ['min', 'max', 'median', 'mean', 'std', 'skew', 'kurtosis', 'entropy']}
    values = np.array(values)
    hist = np.histogram(values, bins=10, density=True)[0] + 1e-9
    return {
        'min': np.min(values),
        'max': np.max(values),
        'median': np.median(values),
        'mean': np.mean(values),
        'std': np.std(values),
        'skew': skew(values),
        'kurtosis': kurtosis(values),
        'entropy': entropy(hist)
    }

def compute_network_features(user_edges):
    features = []
    for user, targets in user_edges.items():
        G = nx.DiGraph()
        G.add_edges_from((user, tgt) for tgt in targets)

        node_count = G.number_of_nodes()
        edge_count = G.number_of_edges()
        out_strength = sum(dict(G.out_degree()).values())
        in_strength = sum(dict(G.in_degree()).values())
        density = nx.density(G)
        clustering = nx.clustering(G.to_undirected(), user) if user in G else 0.0

        out_degrees = [d for n, d in G.out_degree() if n != user]
        in_degrees = [d for n, d in G.in_degree() if n != user]
        strengths = [G.degree(n) for n in G.nodes() if n != user]

        out_stats = compute_distribution_stats(out_degrees)
        in_stats = compute_distribution_stats(in_degrees)
        strength_stats = compute_distribution_stats(strengths)

        feature_row = {
            'user_screen_name': user,
            'network_node_count': node_count,
            'network_edge_count': edge_count,
            'network_out_strength': out_strength,
            'network_in_strength': in_strength,
            'network_density': density,
            'network_clustering_coeff': clustering
        }

        for stat_name, stat_value in out_stats.items():
            feature_row[f'out_strength_{stat_name}'] = stat_value
        for stat_name, stat_value in in_stats.items():
            feature_row[f'in_strength_{stat_name}'] = stat_value
        for stat_name, stat_value in strength_stats.items():
            feature_row[f'strength_{stat_name}'] = stat_value

        features.append(feature_row)

    return pd.DataFrame(features)

def extract_network_features(df):
    user_edges = build_user_interaction_graph(df)
    network_df = compute_network_features(user_edges)
    return network_df


In [ ]:
network_features = extract_network_features(df)

print(network_features.head())
print(network_features.columns)
print(network_features.shape)


/tmp/ipython-input-1978682947.py:43: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  'skew': skew(values),
/tmp/ipython-input-1978682947.py:44: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  'kurtosis': kurtosis(values),


  user_screen_name  network_node_count  network_edge_count  \
0   IllinoisJack85                  77                  76   
1     TheRISEofROD                 359                 358   
2        GmanFan45                 388                 387   
3         MazurikL                 351                 350   
4     zeusFanHouse                 461                 460   

   network_out_strength  network_in_strength  network_density  \
0                    76                   76         0.012987   
1                   358                  358         0.002786   
2                   387                  387         0.002577   
3                   350                  350         0.002849   
4                   460                  460         0.002169   

   network_clustering_coeff  out_strength_min  out_strength_max  \
0                         0                 0                 0   
1                         0                 0                 0   
2                         0        

# **Merge all the features**

In [ ]:
network_features.describe().T[['min', 'max', 'mean', 'std']]

,min,max,mean,std
network_node_count,2.000000e+00,1.781700e+04,2.856829e+01,1.019101e+02
network_edge_count,1.000000e+00,1.781600e+04,2.756829e+01,1.019101e+02
network_out_strength,1.000000e+00,1.781600e+04,2.756829e+01,1.019101e+02
network_in_strength,1.000000e+00,1.781600e+04,2.756829e+01,1.019101e+02
network_density,5.612617e-05,5.000000e-01,1.531051e-01,1.539873e-01
network_clustering_coeff,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
out_strength_min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
out_strength_max,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
out_strength_median,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
out_strength_mean,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [ ]:
unique_authors = df['user_screen_name'].nunique()
print("Unique users (authors):", unique_authors)

user_edges = build_user_interaction_graph(df)
nodes_in_graph = set()
for user, targets in user_edges.items():
    nodes_in_graph.add(user)
    nodes_in_graph.update(targets)

print("Total unique nodes in interaction graph:", len(nodes_in_graph))


Unique users (authors): 60028
Total unique nodes in interaction graph: 462110


In [ ]:
merged_df = user_features.copy()

In [ ]:
merged_df = merged_df.merge(temporal_features, on='user_id', how='left')

In [ ]:
merged_df = merged_df.merge(content_features, on='user_id', how='left')

In [ ]:
merged_df = merged_df.merge(sentiment_features, on='user_id', how='left')

In [ ]:
id_screen_map = df[['user_id', 'user_screen_name']].drop_duplicates()

In [ ]:
network_features_df = network_features.merge(id_screen_map, on='user_screen_name', how='left')

In [ ]:
merged_df = merged_df.merge(network_features_df.drop(columns='user_screen_name'), on='user_id', how='left')

In [ ]:
merged_df = merged_df.drop_duplicates(subset='user_id')

In [ ]:
print("Merged shape:", merged_df.shape)
print("Columns:", merged_df.columns)
print("Any missing values?", merged_df.isna().any().sum())

Merged shape: (60028, 336)
Columns: Index(['id', 'tweet_url', 'created_at', 'user_screen_name', 'text',
       'tweet_type', 'coordinates', 'hashtags', 'media', 'urls',
       ...
       'in_strength_kurtosis', 'in_strength_entropy', 'strength_min',
       'strength_max', 'strength_median', 'strength_mean', 'strength_std',
       'strength_skew', 'strength_kurtosis', 'strength_entropy'],
      dtype='object', length=336)
Any missing values? 135


In [ ]:
len(df['user_id'].unique())

60028

In [ ]:
len(merged_df['user_id'])

60028

In [ ]:
grouped_df = merged_df.sort_values('user_id')

In [ ]:
merged_df

,id,tweet_url,created_at,user_screen_name,text,tweet_type,coordinates,hashtags,media,urls,...,in_strength_kurtosis,in_strength_entropy,strength_min,strength_max,strength_median,strength_mean,strength_std,strength_skew,strength_kurtosis,strength_entropy
0,1267443579045232651,https://twitter.com/IllinoisJack85/status/1267...,2020-06-01 13:11:11,IllinoisJack85,During all those protests people suddenly forg...,original,NaN,NaN,NaN,NaN,...,NaN,2.162327e-08,1.0,1.0,1.0,1.0,0.0,NaN,NaN,2.162327e-08
113,1267886253208473605,https://twitter.com/TheRISEofROD/status/126788...,2020-06-02 18:30:13,TheRISEofROD,"So.... ""Impeachment"" was just a place holder f...",retweet,NaN,NaN,NaN,NaN,...,NaN,2.162327e-08,1.0,1.0,1.0,1.0,0.0,NaN,NaN,2.162327e-08
574,1267332745866477571,https://twitter.com/GmanFan45/status/126733274...,2020-06-01 05:50:47,GmanFan45,Genius @NYCMayor and @andrewcuomo screw up aga...,original,NaN,NaN,NaN,https://t.co/ibqTpxy4f8,...,NaN,2.162327e-08,1.0,1.0,1.0,1.0,0.0,NaN,NaN,2.162327e-08
960,1267400518672044032,https://twitter.com/MazurikL/status/1267400518...,2020-06-01 10:20:05,MazurikL,Who knew that the ‘cure’ to COVID19 was civil ...,retweet,NaN,NaN,http://pbs.twimg.com/tweet_video_thumb/EZY0QJG...,NaN,...,NaN,2.162327e-08,1.0,1.0,1.0,1.0,0.0,NaN,NaN,2.162327e-08
1802,1267269137157558273,https://twitter.com/zeusFanHouse/status/126726...,2020-06-01 01:38:01,zeusFanHouse,Blue States Eliminated Our #Grandparents For C...,original,NaN,Grandparents,NaN,https://t.co/c4sdDzF1dc,...,NaN,2.162327e-08,1.0,1.0,1.0,1.0,0.0,NaN,NaN,2.162327e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2378020,1270244097412849665,https://twitter.com/jetteSee/status/1270244097...,2020-06-09 06:39:27,jetteSee,Why Some Families Are Hit So Hard By COVID-19 ...,original,NaN,NaN,NaN,https://t.co/SzAKaiMicn,...,NaN,2.162327e-08,1.0,1.0,1.0,1.0,0.0,NaN,NaN,2.162327e-08
2378026,1267287016573554688,https://twitter.com/WillResistAdmin/status/126...,2020-06-01 02:49:04,WillResistAdmin,"@DWUhlfelderLaw MAGA no mask, guess we’ll be a...",reply,NaN,WearAMask ProtectOurFirstResponders,NaN,NaN,...,NaN,2.162327e-08,1.0,1.0,1.0,1.0,0.0,NaN,NaN,2.162327e-08
2378093,1267293845995257856,https://twitter.com/VitaminSteve/status/126729...,2020-06-01 03:16:12,VitaminSteve,if a pandemic and a rebellion aren’t moving yo...,retweet,NaN,NaN,http://pbs.twimg.com/media/EZXpI4vWAAM5mX7.jpg,NaN,...,NaN,2.162327e-08,1.0,1.0,1.0,1.0,0.0,NaN,NaN,2.162327e-08
2378133,1267279429031604225,https://twitter.com/Kruzer60913071/status/1267...,2020-06-01 02:18:55,Kruzer60913071,@anacaprana Did you ask your boss if the riote...,reply,NaN,NaN,NaN,https://t.co/UrcX9oEvSe,...,NaN,2.162327e-08,1.0,1.0,1.0,1.0,0.0,NaN,NaN,2.162327e-08


In [ ]:
cols_to_drop = [
    'id', 'tweet_url', 'created_at', 'text', 'text_en', 'tweet_type',
    'in_reply_to_screen_name', 'in_reply_to_status_id', 'in_reply_to_user_id',
    'retweet_or_quote_id', 'retweet_or_quote_screen_name', 'retweet_or_quote_user_id',
    'hashtags', 'media', 'urls', 'favorite_count', 'retweet_count',
    'coordinates', 'place', 'source', 'possibly_sensitive'
]

cleaned_df = grouped_df.drop(columns=[col for col in cols_to_drop if col in grouped_df.columns])




In [ ]:
cleaned_df.head(5)

,user_screen_name,lang,user_id,user_created_at,user_default_profile_image,user_description,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,...,in_strength_kurtosis,in_strength_entropy,strength_min,strength_max,strength_median,strength_mean,strength_std,strength_skew,strength_kurtosis,strength_entropy
2081173,00001745Q,en,00001745Q,2019-07-11 10:56:59,False,http://qmap.pubQAnonUK,11446,2084,3054,2,...,NaN,2.162327e-08,1.0,1.0,1.0,1.0,0.0,NaN,NaN,2.162327e-08
1759675,0000DD02,en,0000DD02,2015-02-17 01:33:29,False,#Constitutionalist #Trump2020 #PrayforAmerica ...,105904,3194,3411,1,...,NaN,2.162327e-08,1.0,1.0,1.0,1.0,0.0,NaN,NaN,2.162327e-08
2051863,001_percent,en,001_percent,2018-10-09 01:42:54,False,woman. multiracial. christian. patriot. horrif...,1038,93,437,0,...,NaN,2.162327e-08,1.0,1.0,1.0,1.0,0.0,NaN,NaN,2.162327e-08
389958,0045James,en,0045James,2020-03-05 02:01:37,False,💯%Trump supporter conservative #maga patriot #...,3387,8869,9607,2,...,NaN,2.162327e-08,1.0,1.0,1.0,1.0,0.0,NaN,NaN,2.162327e-08
1057991,0070American,en,0070American,2015-01-27 00:01:54,False,#MAGA2020 #KAG2020 #QANON,24340,519,846,1,...,NaN,2.162327e-08,1.0,1.0,1.0,1.0,0.0,NaN,NaN,2.162327e-08


In [ ]:
filled_df = cleaned_df.fillna(0)

In [ ]:
filled_df['strength_min'].unique()

array([1., 0.])

In [ ]:
# Save to CSV
filled_df.to_csv("user_level_features_cleaned.csv", index=False)

In [ ]:
filled_df['user_type'].unique()

array(['other', 'bot', 'suspended', 'deleted'], dtype=object)

In [ ]:
import pandas as pd
import numpy as np


df = pd.read_csv("user_level_features_cleaned.csv")

df = df[~df['user_type'].isin(['suspended', 'deleted'])]

bool_cols = df.select_dtypes(include='bool').columns.tolist()

df[bool_cols] = df[bool_cols].astype(int)

for col in df.columns:
    if df[col].dtype == 'object' and df[col].dropna().isin(['True', 'False']).all():
        df[col] = df[col].map({'True': 1, 'False': 0})

X = df.select_dtypes(include=[np.number]).drop(columns=['user_type'], errors='ignore')
y = df['user_type'].map({'other': 0, 'bot': 1})


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier( n_estimators=100,
    criterion='gini',
    random_state=42,
    n_jobs=-1)
clf.fit(X_train, y_train)


RandomForestClassifier(n_jobs=-1, random_state=42)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


y_pred = clf.predict(X_test)
y_proba = clf.predict_proba(X_test)[:, 1]


print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_proba))


Accuracy: 0.947331361780031
Precision: 0.9263803680981595
ROC AUC: 0.9335567592326292


# **calculating the agreement score**

In [ ]:
import pandas as pd

original_df = pd.read_csv(data)

user_ids = df['user_id'].values
user_features = X.copy()
user_predictions = clf.predict(user_features)
user_type_map = dict(zip(user_ids, user_predictions))


original_df['predicted_user_type'] = 0

original_df.loc[original_df['user_id'].isin(user_type_map),
                'predicted_user_type'] = original_df['user_id'].map(user_type_map)

post_level_df = original_df[['id', 'predicted_user_type']]


/tmp/ipython-input-3738877880.py:3: DtypeWarning: Columns (6,11,15,30,34) have mixed types. Specify dtype option on import or set low_memory=False.
  original_df = pd.read_csv(data)


In [ ]:
post_level_df

,id,predicted_user_type
0,1267443579045232651,1
1,1267520253904510976,1
2,1267587387443593217,1
3,1267840580299694082,1
4,1267887896884908032,1
...,...,...
1716721,1277568630587363330,0
1716722,1277573029749555200,0
1716723,1277761285028024320,0
1716724,1278001200026333185,0


In [ ]:
botometer_df=pd.read_csv("/content/drive/MyDrive/Nour/twitter/Botometer_classified_posts.csv")

In [ ]:
post_level_df

,id,predicted_user_type
0,1267443579045232651,1
1,1267520253904510976,1
2,1267587387443593217,1
3,1267840580299694082,1
4,1267887896884908032,1
...,...,...
1716721,1277568630587363330,0
1716722,1277573029749555200,0
1716723,1277761285028024320,0
1716724,1278001200026333185,0


In [ ]:
botometer_df

,id,post_type
0,1267443579045232651,bot
1,1267520253904510976,bot
2,1267587387443593217,bot
3,1267840580299694082,bot
4,1267887896884908032,bot
...,...,...
1716721,1277568630587363330,other
1716722,1277573029749555200,other
1716723,1277761285028024320,other
1716724,1278001200026333185,other


In [ ]:
len(post_level_df)

1716726

In [ ]:
len(botometer_df)

1716726

In [ ]:
botometer_df = botometer_df.rename(columns={"post_type": "botometer"})

In [ ]:
post_level_df

,id,predicted_user_type
0,1267443579045232651,1
1,1267520253904510976,1
2,1267587387443593217,1
3,1267840580299694082,1
4,1267887896884908032,1
...,...,...
1716721,1277568630587363330,0
1716722,1277573029749555200,0
1716723,1277761285028024320,0
1716724,1278001200026333185,0


In [ ]:
merged_df = pd.merge(botometer_df, post_level_df, on="id", how="inner")

In [ ]:
merged_df["botometer"] = merged_df["botometer"].apply(lambda x: 1 if x == "bot" else 0)

In [ ]:
merged_df

,id,botometer,predicted_user_type
0,1267443579045232651,1,1
1,1267520253904510976,1,1
2,1267587387443593217,1,1
3,1267840580299694082,1,1
4,1267887896884908032,1,1
...,...,...,...
1716721,1277568630587363330,0,0
1716722,1277573029749555200,0,0
1716723,1277761285028024320,0,0
1716724,1278001200026333185,0,0


In [ ]:
from sklearn.metrics import cohen_kappa_score

kappa = cohen_kappa_score(merged_df['botometer'], merged_df['predicted_user_type'])

print(f"Cohen's Kappa agreement score: {kappa:.4f}")

Cohen's Kappa agreement score: 0.8097


In [2]:
import pandas as pd
df = pd.read_csv("user_level_features_cleaned.csv")


In [3]:
df.columns

Index(['user_screen_name', 'lang', 'user_id', 'user_created_at',
       'user_default_profile_image', 'user_description',
       'user_favourites_count', 'user_followers_count', 'user_friends_count',
       'user_listed_count',
       ...
       'in_strength_kurtosis', 'in_strength_entropy', 'strength_min',
       'strength_max', 'strength_median', 'strength_mean', 'strength_std',
       'strength_skew', 'strength_kurtosis', 'strength_entropy'],
      dtype='object', length=315)

In [5]:
df.shape

(26481, 315)